# Python日志分割

In [1]:
import logging
from logging.handlers import BaseRotatingHandler

In [2]:
class MultiProcessSafeDailyRotatingFileHandler(BaseRotatingHandler):

    def __init__(self, filename, suffix='%Y-%m-%d', encoding=None, delay=False, utc=False, **kwargs):
        self.utc = utc
        self.suffix = suffix
        self.baseFilename = filename
        self.currentFileName = self._compute_fn()
        BaseRotatingHandler.__init__(self, filename, 'a', encoding, delay)

    def shouldRollover(self, record):
        if self.currentFileName != self._compute_fn():
            return True
        return False

    def doRollover(self):
        if self.stream:
            self.stream.close()
            self.stream = None
        self.currentFileName = self._compute_fn()

    def _compute_fn(self):
        return self.baseFilename + "." + time.strftime(self.suffix, time.localtime())

    def _open(self):
        if self.encoding is None:
            stream = open(self.currentFileName, self.mode)
        else:
            stream = codecs.open(self.currentFileName, self.mode, self.encoding)
        return stream


In [3]:
import os
import socket

In [4]:
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
HOST_NAME = socket.gethostname()

In [5]:
LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': True,
    'formatters': {
       'generic': {
            "format": "%(asctime)s - (%(name)s)[%(levelname)s][%(host)s]: "
                    + "%(request)s %(message)s %(status)d %(byte)d",
            "datefmt": "[%Y-%m-%d %H:%M:%S %z]",
            "class": "logging.Formatter",
       }
    },
    'filters': {
    },
    'handlers': {
        'default': {
            'level': 'DEBUG',
            'class': 'MultiProcessSafeDailyRotatingFileHandler',  # logger class module
            'filename': os.path.join(BASE_DIR, 'uwsgi_output', HOST_NAME + '.log'),
            'formatter': 'generic',
        },
    },
    'loggers': {
        'root': {
            'handlers': ['default'],
            'level': 'INFO',
            'propagate': False
        }
    }
}

In [6]:
# import logging
# logging.config.dictConfig(LOGGING_CONFIG)